In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [3]:
# В зоне обслуживания

In [4]:
nvis = pd.read_csv("number_visits2.txt", sep="\t")
data = pd.read_csv("база данных.csv", sep=",")
new = pd.read_csv("new_info.tsv", sep="\t")

In [5]:
new['year'] = new['Дата открытия'].str.extract('(\d{4})')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [6]:
nvis = nvis.fillna('')
new = new.fillna('')
data = data.fillna('')

In [7]:
nvis = nvis[:-1]

In [8]:
nvis.shape

(9546, 4)

In [9]:
set(nvis.prep)

{'', 'в', 'за', 'на', 'около', 'от', 'с', 'свыше'}

# Посетителей в месяц

### в n году/ за n год

In [10]:
a = nvis['n_days'].loc[nvis['prep'].isin(["в", "за"]) & (nvis['year'] != "")].index
nvis['n_days'].loc[a] = [365] * len(a)

### на n год

In [11]:
ind = nvis.loc[(nvis['prep'] == "на") & (nvis['year'] != "")].index
ind = set(ind)

In [12]:
ind2 = set(new.loc[new['year'] != ''].index)

In [13]:
comb = ind2 & ind

In [14]:
a = nvis['year'].loc[comb].apply(int)
b = new['year'].loc[comb].apply(int)

res = a - b

In [15]:
nvis['n_days'].loc[comb] = res * 365

### с n года

In [16]:
ind = nvis.loc[(nvis['prep'] == "с") & (nvis['year'] != "")].index
res = 2019 - nvis['year'].loc[ind].apply(int)
nvis['n_days'].loc[ind] = res * 365

In [39]:
a = nvis.loc[(nvis['n_days'].apply(type) != int) & (nvis['n_days'] != "")]
nvis['n_days'].loc[a.index] = a['n_days'].apply(int)

In [17]:
df = nvis[nvis['n_visits'].str.contains("_")]

In [18]:
df2 = nvis[nvis['n_visits'].str.contains(",")]

In [19]:
ind = set(df2.index) - set(df.index)
nvis['n_visits'].loc[ind] += '_1000'

In [20]:
nvis['n_visits'] = nvis.n_visits.str.replace(' ', '')
nvis['n_visits'] = nvis.n_visits.str.replace(',', '.')

In [21]:
df = nvis[nvis['n_visits'].str.contains("_")]

In [22]:
nvis['n_visits'].loc[ind]

3210    4.5_1000
Name: n_visits, dtype: object

In [23]:
str_num = {'полтора':1.5,
              'два':2,
              'три':3,
              'четыре':4,
              'пять':5,
              'шесть':6,
              'семь':7,
              'восемь':8,
              'девять':9,
              'десять':10,
             }

def str_to_num(x):
    
    global str_num
    
    p = morph.parse(x)[0]
    return str_num.get(p.normal_form)



In [24]:
for index in df.index:
    
    number, z = df.loc[index].n_visits.split('_')
    z = int(z)
    
    if '.' in number:
        n = float(number)
        nvis['n_visits'][index] = n * z
        
    elif number.isalpha():
        n = str_to_num(number)
        nvis['n_visits'][index] = n * z
    
    else:
        n = int(number)
        if n/z > 1: nvis['n_visits'][index] = n
        else: nvis['n_visits'][index] = n * z

3287
4394
5769
6071
7333
8200


In [25]:
a = np.where((nvis['n_visits'].apply(type) == str) & (nvis['n_visits'] != ''))
nvis['n_visits'].loc[a] = nvis['n_visits'].loc[a].apply(int)

In [26]:
nvis['nvis/month'] = [0]* nvis.shape[0]

In [40]:
a = np.where((nvis['n_visits'].apply(type) == int) & (nvis['n_days'].apply(type) == int))
b = nvis.loc[a]

nvis['nvis/month'].loc[a] = (b['n_visits'] / b['n_days']) * 30

# All together

In [32]:
result = pd.concat([nvis, data, new])
result = result.fillna('')

In [33]:
result.keys()

Index(['Hosting', 'ID места ЕИПСК', 'ServiceName', 'Start', 'Timezone',
       'Types[]', 'Unnamed: 0', 'n_days', 'n_visits', 'nvis/month', 'prep',
       'Адрес', 'Адрес сайта', 'Адрес электронной почты', 'Адрес.1',
       'Аудитория', 'ВС', 'ВТ', 'Виртуальный тур', 'Госкаталог',
       'Дата открытия', 'Дата последнего обновления записи',
       'Дата создания записи', 'Дополнительные изображения', 'ЕИПСК', 'ИНН',
       'Идентификатор ЕИПСК', 'Изба-читальня', 'Изображение', 'Категория',
       'Категория учреждения', 'Количество посетителей', 'Культура.РФ',
       'Местоположение', 'Местоположение.1', 'На карте', 'На карте.1',
       'Название', 'Название.1', 'Номер', 'Описание', 'ПН', 'ПТ', 'Подчинение',
       'Примечание', 'Примечание.1', 'Принадлежность',
       'Профессиональный уровень', 'Размер фонда', 'СБ', 'СР', 'Статистика',
       'Тип библиотеки', 'Тип категории', 'Тэги', 'Уверенность в фонде',
       'Улица', 'Улица.1', 'ЧТ', 'Часовой пояс', 'Часовой пояс.1',
       'Эл